In [320]:
#uncomment to install selenium
#conda install -c conda-forge selenium
#!pip install fake_useragent

  Using cached fake-useragent-0.1.11.tar.gz (13 kB)
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13487 sha256=63df878f545a07ae8918ab6cb4d354c45c9ef147a0d62ad1761168d9609ab04b
  Stored in directory: /Users/chrismalasics/Library/Caches/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent
You should consider upgrading via the '/Users/chrismalasics/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
from bs4 import BeautifulSoup
import requests
import time, os
import re
import random
from random import randint
import random
from time import sleep

import pandas as pd
from pymongo import MongoClient
from pprint import pprint
import json

In [361]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [322]:
from fake_useragent import UserAgent

### Create Mongo collection for urls

In [4]:
client = MongoClient()

In [5]:
db = client['philadelphia_home_prices']

In [6]:
db.list_collection_names()

['prices_messy', 'listing_urls']

In [7]:
#uncomment to delete the collection
#db.drop_collection('listing_urls')

{'nIndexesWas': 1, 'ns': 'philadelphia_home_prices.listing_urls', 'ok': 1.0}

In [8]:
db.list_collection_names()

['prices_messy']

In [9]:
col = db.create_collection('listing_urls')

In [10]:
db.list_collection_names()

['prices_messy', 'listing_urls']

### scrape pages for listing urls

In [11]:
url = 'https://www.realtor.com/'

In [16]:
# 44 listings per page. Choose start and stop page
page_from = 21
page_to = 25

In [13]:
col.count()

/Users/chrismalasics/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


0

In [17]:
for i in range(page_from, page_to+1):
    #print(i) #prints page that is open in case bot is detected
    driver = webdriver.Chrome(chromedriver)
    driver.get(url + 'soldhomeprices/Philadelphia_PA/pg-' + str(i))
    
    sleep(random.uniform(60,120))
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    page = str(soup.find_all('div', class_='photo-wrap'))
    extensions = re.findall(r'[A-z]+-[A-z]+\/[0-9-A-z]+[^"]', page)
    
    if len(extensions) == 0:
        print('bot is caught on page {}'.format(i))
        break
        
    for i in extensions:
        page_url = url + i
        page = {'url': page_url}
        col.insert_one(page)
        
    sleep(random.uniform(1,3))
    
    driver.close()

In [18]:
col.count()

/Users/chrismalasics/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


1100

### Pull urls from mongoDB into memory

In [49]:
#pull urls from mongo db
cursor = db.listing_urls.find({})
listings = list(cursor)

In [50]:
#convert dictionary values to list
listing_urls_list = []
for i in listings:
    url = i['url']
    listing_urls_list.append(url)

In [51]:
#see first 5 in list
listing_urls_list[0:5]

['https://www.realtor.com/realestateandhomes-detail/1232-N-Mascher-St-Apt-A_Philadelphia_PA_19122_M49182-95881',
 'https://www.realtor.com/realestateandhomes-detail/212-Carpenter-St-Rear-C_Philadelphia_PA_19147_M34206-29412',
 'https://www.realtor.com/realestateandhomes-detail/1211-E-Cheltenham-Ave_Philadelphia_PA_19124_M33700-20336',
 'https://www.realtor.com/realestateandhomes-detail/1002-Brandywine-St_Philadelphia_PA_19123_M39147-93479',
 'https://www.realtor.com/realestateandhomes-detail/2607-N-31st-St_Philadelphia_PA_19132_M32883-74202']

### Get data from urls

#### new collection for listing data

In [291]:
col = db.create_collection('listing_data')

In [292]:
db.collection_names()

/Users/chrismalasics/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


['prices_messy', 'listing_urls', 'listing_data']

In [293]:
#uncomment to drop
#db.drop_collection('listing_data')

In [294]:
db.collection_names()

/Users/chrismalasics/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


['prices_messy', 'listing_urls', 'listing_data']

In [365]:
listing_from = 25
listing_to = len(listing_urls_list)
listing_data = {}

In [366]:
for i in range(listing_from, listing_to+1): 
    
    print(i, listing_urls_list[i])
    
    #ua = UserAgent()
    #user_agent = {'User-agent': ua.random}
    #print(user_agent)

    #response  = requests.get(listing_urls_list[i], headers = user_agent)
    
    #driver = webdriver.Chrome(chromedriver)
    #driver.get(listing_urls_list[i])
    
    if i//5 == 0:
        sleep(random.uniform(60,120))
        
    ####     
    #session=requests.Session()
    #session.headers = user_agent#{'User-Agent': ua.random} #'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    #print(user_agent)
    ##base_url='https://www.realtor.com'
    ##re_list=[]
    ##realtor_default_count=44
    ##page_counter=1
    ##search_url=''.join([base_url, '/realestateandhomes-search/Sacramento_CA'])
    #page=session.get(listing_urls_list[i])
    #c=page.content
    #soup=BeautifulSoup(c,"html.parser")
    ####
    
    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    print(userAgent)
    options.add_argument(f'user-agent={userAgent}')
    driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)
    driver.get(listing_urls_list[i])

    ###
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    soup_to_str = str(soup)
    
    street_address = (soup.find('span', itemprop="streetAddress")).text
    address_locality = (soup.find('span', itemprop="addressLocality")).text
    address_region = (soup.find('span', itemprop="addressRegion")).text
    postal_code = (soup.find('span', itemprop="postalCode")).text

    price = re.findall(r'(?<=Last Sold for \$)\d{0,3},{0,1}\d{1,3},{0,1}\d{1,3}', soup_to_str)
    days_on_market = re.findall(r'(\d{1,3})[\s]*</div>\n<span>Days on market', soup_to_str)
    sqft_house = re.findall(r'(\d{0,2},{0,1}\d{3})</span>[<>\s\/A-z]+sq\sft',soup_to_str)
    sqft_lot = re.findall(r'lot_size":([\d.]+[^,])', soup_to_str)
    full_bath = re.findall(r'Full\s{1}Bathrooms:\s{1}(\d{1,2})',soup_to_str)
    half_bath = re.findall(r'1\/2\sBathrooms:([0-9\s]+[^<])',soup_to_str)
    bedrooms = re.findall(r'Bedrooms:\s{1}(\d{1,2})',soup_to_str)
    garage = re.findall(r'Garage\sSpaces:([\d\s]+[^<])', soup_to_str)
    master_bath = re.findall(r'Master\sBath(s):\s([A-Z\s]+[^<])', soup_to_str)

    cooling = re.findall(r'Cooling\sFeatures:(\s[A-z\s\/\(\)]+[^<])', soup_to_str)
    heating = re.findall(r'Heating\sFeatures:(\s[A-z\s\/\(\)]+[^<])', soup_to_str)

    annual_tax = re.findall(r'Annual Tax Amount:\s(\d{1,6}).', soup_to_str)
    house_type = re.findall(r'Structure\sType:([\sA-z\/-]+[^<])',soup_to_str)
    built = re.findall(r'[Bb]uilt:(\s+\d+)', soup_to_str)
    updated = re.findall(r'Date\supdated:([0-9\s\/]+[^<])', soup_to_str)

    neighborhood = re.findall(r'Source Neighborhood: ([A-z\s]*)', soup_to_str)
    neighborhood_median_sales = re.findall(r'(\d{0,3},{0,1}\d{0,3},{0,1}\d{0,3})[\D]*Median Sales Price', soup_to_str)
    neighborhood_median_days_on_market = re.findall(r'(\d{1,3})[\D]*Median Days on Market', soup_to_str)
    neighborhood_price_SQFT = re.findall(r'(\d{0,3},{0,1}\d{0,3})[\D]*Price Per Sq Ft', soup_to_str)
    association = re.findall(r'Association: ([A-z]{2,3})', soup_to_str)
    association_monthly = re.findall(r'Monthly Association Fees:\s(\d{1,5})', soup_to_str)

    listing_data = {'Address': street_address,
                    'Locality': address_locality,
                    'Region': address_region,
                    'Postal Code': postal_code,
                    'Price': price,
                    'Days on Market': days_on_market,
                    'SQFT House': sqft_house,
                    'SQFT Lot': sqft_lot,
                    'Full Baths': full_bath,
                    'Half Baths': half_bath,
                    'Bedrooms': bedrooms,
                    'Garage': garage,
                    'Master Bath': master_bath,
                    'Cooling': cooling,
                    'Heating': heating,
                    'Annual Tax': annual_tax,
                    'House Type': house_type,
                    'Build Year': built,
                    'Updated': updated,
                    'Neighborhood': neighborhood,
                    'Neighborhood median sales': neighborhood_median_sales,
                    'Neighborhood Median DOM' : neighborhood_median_days_on_market,
                    'Neighborhood Price SQFT': neighborhood_price_SQFT,
                    'Association': association,
                    'Association Monthly': association_monthly}

    driver.close()
    
    #insert listing data into mongo database
    col.insert_one(listing_data)
    print(listing_data)
    
    sleep(random.uniform(120,180)) #pause

25 https://www.realtor.com/realestateandhomes-detail/5254-N-Howard-St_Philadelphia_PA_19120_M35069-57480
Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36


/Users/chrismalasics/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: DeprecationWarning: use options instead of chrome_options


{'Address': '5254 N Howard St,', 'Locality': 'Philadelphia', 'Region': 'PA', 'Postal Code': '19120', 'Price': ['161,650', '161,650', '161,650', '161,650', '161,650', '161,650', '161,650', '161,650', '161,650'], 'Days on Market': ['55'], 'SQFT House': ['1,230', '1,230', '1,230', '1,230'], 'SQFT Lot': ['1050', '1050'], 'Full Baths': ['2'], 'Half Baths': [], 'Bedrooms': ['3'], 'Garage': [], 'Master Bath': [], 'Cooling': [' Window Unit(s)'], 'Heating': [' Radiator'], 'Annual Tax': ['1307'], 'House Type': [' Interior Row/Townhouse'], 'Build Year': [' 1925', ' 1925'], 'Updated': [' 08/07/2020'], 'Neighborhood': ['OLNEY'], 'Neighborhood median sales': ['123,000'], 'Neighborhood Median DOM': ['67'], 'Neighborhood Price SQFT': ['113'], 'Association': ['No'], 'Association Monthly': ['0'], '_id': ObjectId('5f9b1a79329f96e5db63d285')}
26 https://www.realtor.com/realestateandhomes-detail/1307-S-17th-St_Philadelphia_PA_19146_M39955-53546
Mozilla/5.0 (Windows NT 6.1; WOW64; rv:21.0) Gecko/20100101 Fi

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
len(extensions)

In [ ]:
listing_url_page1

In [ ]:
listings

In [ ]:
soup_to_str

In [ ]:
listing_url_page1

In [ ]:
driver.page_source(:1000)

In [ ]:
my_randoms = random.sample(range(44), 3)

In [ ]:
my_randoms

In [ ]:
rand

In [ ]:
extensions

In [ ]:
url

In [ ]:
re.search(r'Cooling\sFeatures:\s[A-z\s\/,]+[^<]', soup_to_str)

In [ ]:
re.findall(r'Heating\sFeatures:\s[A-z\s,-]+[^<]', soup_to_str)

In [ ]:
extensions[0]

In [ ]:
(soup.find('span', class_='data-price')).text

In [ ]:
soup.find('span', class_='data-price')

In [ ]:
soup

In [314]:
listing_urls_list[0:5]

['https://www.realtor.com/realestateandhomes-detail/1232-N-Mascher-St-Apt-A_Philadelphia_PA_19122_M49182-95881',
 'https://www.realtor.com/realestateandhomes-detail/212-Carpenter-St-Rear-C_Philadelphia_PA_19147_M34206-29412',
 'https://www.realtor.com/realestateandhomes-detail/1211-E-Cheltenham-Ave_Philadelphia_PA_19124_M33700-20336',
 'https://www.realtor.com/realestateandhomes-detail/1002-Brandywine-St_Philadelphia_PA_19123_M39147-93479',
 'https://www.realtor.com/realestateandhomes-detail/2607-N-31st-St_Philadelphia_PA_19132_M32883-74202']